In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [25]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [40]:
# Set up API keys
OPENAI_API_KEY = os.getenv(key="OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv(key="PINECONE_API_KEY")

In [41]:
# Load the Pinecone vector store
index_name = "hospital-rag"

vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

In [42]:
# Set up LLM, using GPT-4o-mini with temp 0.0
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [31]:
_template = """
You are a helpful AI assistant for answering questions based on the provided documents.
Your goal is to provide accurate and concise answers.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know. DO NOT try to make up an answer.
Keep the answer concise.
And you need to use same language with user's questions.

CONTEXT:
{context}

CHAT HISTORY:
{chat_history}

QUESTION:
{question}

Answer in the same language as the question.
Helpful Answer:"""

In [32]:
QA_PROMPT = PromptTemplate(
    template=_template,
    input_variables=["context", "chat_history", "question"]
)

In [43]:
# Set up conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

In [44]:
# Set up Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(k=4),
    memory=memory,
    return_source_documents=True, # Set to True to see which documents were retrieved
    combine_docs_chain_kwargs={"prompt": QA_PROMPT},
    verbose=False # Set to True to see the inner workings of the chain
)

In [46]:
# Chat loop inside Jupyter cell
while True:
    user_query = input("\nYou: ")
    if user_query.lower() in ["exit", "quit"]:
        print(f"User: {user_query}")
        print("Chatbot: Goodbye!")
        break

    result = qa_chain({"question": user_query})

    print(f"User: {user_query}")
    print(f"Chatbot: {result['answer']}")

    # Optionally, print the source documents that were used
    print("\n--- Source Documents ---")
    for doc in result['source_documents']:
        print(f"- Page {doc.metadata.get('page_number', 'N/A')}: {doc.page_content}...")
    print("------------------------\n")

User: My name is Van Sy, what treatments did Mrs. Brandy Flowers receive during her hospital stay?
Chatbot: Mrs. Brandy Flowers received a combination of inhaled bronchodilators (such as albuterol) and corticosteroids to manage airway inflammation. Pulmonary function tests were conducted to assess lung function, and antibiotics were administered if a bacterial infection was suspected. She was also advised on lifestyle modifications, including smoking cessation and environmental control.

--- Source Documents ---
- Page N/A: 
    Visit ID: 4
    Patient: Mrs. Brandy Flowers, Sex: Male, DOB: 1989-01-26, Blood Type: AB+
    Physician: Mark Vang (Graduated: 1987-08-11 from University of California, San Francisco School of Medicine)
    Hospital: Vaughn PLC, State: CO
    Complaint: Persistent cough and shortness of breath
    Diagnosis: J45.909 - Unspecified asthma, uncomplicated
    Treatment: Prescribed a combination of inhaled bronchodilators (such as albuterol) and corticosteroids to m